In [3]:
import numpy as np
import cv2 as cv

# Inspección de imagen

In [4]:
class ImageInspector:
	def __init__(self, image, title):
		self.image = image
		self.title = title
		self.eyedrop_col = None
		cv.namedWindow(title)

	def show(self):
		image = self.image.copy()
		if self.eyedrop_col is not None:
			g, b, r = self.eyedrop_col
			image = cv.putText(image, f"({r}, {g}, {b})", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
		cv.imshow(self.title, image)

	def register_cb(self):
		def cb(event, x, y, *_):
			if event == cv.EVENT_MOUSEMOVE:
				self.eyedrop_col = self.image[y, x]
				self.show()
		cv.setMouseCallback(self.title, cb)


In [ ]:
img = ImageInspector(cv.imread("tears.png"), "tears")

img.register_cb()

img.show()

cv.waitKey(0)

cv.destroyAllWindows()

# Editor

In [14]:
class LineTool:
	def __init__(self):
		self.point = None

	@staticmethod
	def icon(w, h, status, col, thickness):
		ico = np.zeros((w, h, 3), dtype=np.uint8)
		match status:
			case "normal":
				ico[:,:,:] = np.repeat(200, 3)
			case "hover":
				ico[:,:,:] = np.repeat(180, 3)
			case "focus":
				ico[:,:,:] = np.repeat(150, 3)
		tl = (
			int(w * .25),
			int(h * .25)
		)
		br = (
			int(w * .75),
			int(h * .75)
		)
		ico = cv.line(ico, tl, br, col, thickness)
		return ico

	def preview(self, img, mouse, col, thickness):
		if self.point is None:
			return img
		return cv.line(img, self.point, mouse, col, thickness)

	def commit(self, img, mouse, col, thickness):
		if self.point is None:
			self.point = mouse
			return None
		img = cv.line(img, self.point, mouse, col, thickness)
		self.point = None
		return img

class RectangleTool:
	def __init__(self):
		self.point = None
	
	@staticmethod
	def icon(w, h, status, col, thickness):
		ico = np.zeros((w, h, 3), dtype=np.uint8)
		match status:
			case "normal":
				ico[:,:,:] = np.repeat(200, 3)
			case "hover":
				ico[:,:,:] = np.repeat(180, 3)
			case "focus":
				ico[:,:,:] = np.repeat(150, 3)
		tl = (
			int(w * .25),
			int(h * .25)
		)
		br = (
			int(w * .75),
			int(h * .75)
		)
		ico = cv.rectangle(ico, tl, br, col, thickness)
		return ico

	def preview(self, img, mouse, col, thickness):
		if self.point is None:
			return img
		top_left = (
			min(self.point[0], mouse[0]),
			min(self.point[1], mouse[1])
		)
		bottom_right = (
			max(self.point[0], mouse[0]),
			max(self.point[1], mouse[1])
		)
		return cv.rectangle(img, top_left, bottom_right, col, thickness)

	def commit(self, img, mouse, col, thickness):
		if self.point is None:
			self.point = mouse
			return None
		top_left = (
			min(self.point[0], mouse[0]),
			min(self.point[1], mouse[1])
		)
		bottom_right = (
			max(self.point[0], mouse[0]),
			max(self.point[1], mouse[1])
		)
		img =  cv.rectangle(img, top_left, bottom_right, col, thickness)
		self.point = None
		return img

class CircleTool:
	def __init__(self):
		self.center = None
	
	@staticmethod
	def icon(w, h, status, col, thickness):
		ico = np.zeros((w, h, 3), dtype=np.uint8)
		match status:
			case "normal":
				ico[:,:,:] = np.repeat(200, 3)
			case "hover":
				ico[:,:,:] = np.repeat(180, 3)
			case "focus":
				ico[:,:,:] = np.repeat(150, 3)
		center = (w//2, h//2)
		radius = int(min(w, h) // 4)
		ico = cv.circle(ico, center, radius, col, thickness)
		return ico

	def preview(self, img, mouse, col, thickness):
		if self.center is None:
			return img
		diff = np.array(self.center) - np.array(mouse)
		r = int(np.sqrt((diff*diff).sum()))
		return cv.circle(img, self.center, r, col, thickness)

	def commit(self, img, mouse, col, thickness):
		if self.center is None:
			self.center = mouse
			return None
		diff = np.array(self.center) - np.array(mouse)
		r = int(np.sqrt((diff*diff).sum()))
		img =  cv.circle(img, self.center, r, col, thickness)
		self.center = None
		return img

TOOL_WIDTH = 70
TOOL_PAD = 5

class ImageEditor:

	tools = [
		LineTool,
		RectangleTool,
		CircleTool,
	]

	def __init__(self, image, title):
		self.image = image
		self.title = title
		self.color = np.array([0, 0, 0], dtype=np.uint8)
		self.thickness = 2
		self.tool = None
		self.hover = None
		cv.namedWindow(title)
		for idx, c in zip([2, 1, 0], "rgb"):
			def col_tb(x, idx=idx):
				self.color[idx] = x
				self.show()
			cv.createTrackbar(c, title, 0, 255, col_tb)
		def thickness_tb(x):
			self.thickness = x
			self.show()
		cv.createTrackbar("thickness", title, self.thickness, 20, thickness_tb)
		def mouse_cb(event, x, y, *_):
			col = (int(self.color[0]), int(self.color[1]), int(self.color[2]))
			img_pos = (x - TOOL_WIDTH, y)
			self.hover = None
			if event == cv.EVENT_MOUSEMOVE:
				if x < TOOL_WIDTH:
					tool_idx = y // (TOOL_WIDTH + TOOL_PAD)
					tool_ypos = y % (TOOL_WIDTH + TOOL_PAD)
					if tool_idx < len(self.tools) and tool_ypos <= TOOL_WIDTH:
						self.hover = self.tools[tool_idx]
						self.show()
				elif self.tool:
					self.show(self.tool.preview(self.image.copy(), img_pos, col, self.thickness))
			if event == cv.EVENT_LBUTTONDOWN:
				if x < TOOL_WIDTH:
					tool_idx = y // (TOOL_WIDTH + TOOL_PAD)
					tool_ypos = y % (TOOL_WIDTH + TOOL_PAD)
					if tool_idx < len(self.tools) and tool_ypos <= TOOL_WIDTH:
						self.tool = self.tools[tool_idx]()
						self.show()
				elif self.tool is not None:
					res = self.tool.commit(self.image, img_pos, col, self.thickness)
					if res is not None:
						self.image = res
						self.show()
		cv.setMouseCallback(title, mouse_cb)


	def show(self, img=None):
		if img is None:
			img = self.image
		imh, imw = img.shape[:2]
		ww = imw + TOOL_WIDTH
		wh = max(imh, len(self.tools) * (TOOL_WIDTH + TOOL_PAD) - TOOL_PAD)
		window = np.zeros((wh, ww, 3), dtype=np.uint8)
		window[:imh, TOOL_WIDTH:ww, :] = img[:,:,:]
		col =(int(self.color[0]), int(self.color[1]), int(self.color[2]))
		for i, tool in enumerate(self.tools):
			state = "normal"
			if self.hover == tool:
				state = "hover"
			if isinstance(self.tool, tool):
				state = "focus"
			ico = tool.icon(TOOL_WIDTH, TOOL_WIDTH, state, col, self.thickness)
			start = (TOOL_WIDTH + TOOL_PAD) * i
			window[start:start+TOOL_WIDTH, :TOOL_WIDTH, :] = ico
		cv.imshow(self.title, window)

	def run(self):
		self.show()
		cv.waitKey(0)
		cv.destroyWindow(self.title)

In [15]:
ed = ImageEditor(cv.imread("tears.png"), "tears")
ed.run()

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is not the object's thread (0x144176f0).
Cannot move to target thread (0x14364080)

QObject::moveToThread: Current thread (0x14364080) is n